# Freedmen's Bureau Analysis 

## Task #0: Configure an LLM

### Import libraries that we'll use in this notebook

In [30]:
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_openai import OpenAI, ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.schema import HumanMessage, SystemMessage, AIMessage

import json


## Task #1: Parse Entitities from Simple Examples (Warmup)

### Samples to be able to parse correctly

In [40]:
inputs = [
    "John James agrees to pay $50/month to RJ Hampshire for work on the Farm",
    "Elizabeth James will pay $30 per month to Levi Rodgers for Gardening",
    """This Agreement made this 14th day of August A.D. 1865, by and between F.R.J. Terry of the county of Copiah and State of Mississippi of the first part, and the person hereinafter named and undersigned, Freedmen of the second part [[?]] That for the purpose of working in the [[?]] known as Beagley's [[?]] Yard in the county aforesaid for two months commencing on the 14th day of August 1865 and terminating on the 14th day of October 1865. The said F.R.J. Terry party of the first part, in consideration of the [[?]] and conditions hereinafter mentioned on the part of the party of the second part agrees to pay said laborer "10" ten dollars per month and furnish free of charge clothing and good of good quality and sufficient quantity, good and sufficient quarters, and kind and humane treatment. And it is further agreed that in case the said F.R.J. Terry shall fail, neglect, or refuse to fulfill any of the obligations assumed by him, he shall besides the legal recourse left to the party aggrieved render this contract liable to amendment by the Provost Marshal of Freedmen. And it is agreed on the part of the party of second part that he will well and faithfully perform such labor as the said F.R.J. Terry may require of him for the time aforesaid, nor exceeding ten hours per day in summer and nine hours in winter. And in case the said laborer shall absent himself from or refuse to perform the labor herein promised, he shall loose the time and be punished as such manner as the Provost Marshal shall deem propper."""
]

outputs = [
    {"payer": "John James", "payee": "RJ Hampshire", "amount": 50, "pay frequency": "monthly", "description": "farming"},
    {"payer": "Elizabeth James", "payee": "Levi Rodgers", "amount": 30, "pay frequency": "monthly", "description": "gardening"},
    {"payer": "F.R.J. Terry", "payee": "Freedmen", "amount":10, "pay frequency": "monthly", "description": "working in the yard"}
]

### Your code here

In [41]:
system_message = """
  From the following historical document text, please grab out the following items and return with the following format
  [{{
    "payer": "<Payer Name>",
    "payee": "<Payee Name>",
    "amount": <Amount Paid>, # This is the amount of money in USD
    "pay frequency": "<Frequency Paid>" # Only options are "monthly", "weekly", "yearly", "one-time", or "unknown"
    "description": "<Description of Work>" # This is the description of the work being done
  }},
  {{<ITEM 2>}}, {{Item 3>}}, ...]

  So for the following text:
  "Jane Doe will pay $50/month to John Smith for work on the Farm"

  the output text generated would be:
  [{{
    "payer": "Jane Doe",
    "payee": "John Smith",
    "amount": 50,
    "pay frequency": "monthly"
    "description": "work on the Farm"
  }}]

  IMPORTANT: Return only the list of Python dictionary objects and nothing more. It is possible there are no matches, in which case return an empty list.
  """

In [44]:
chat = ChatOpenAI()
# chat = ChatOllama(model="llama2")

messages = [SystemMessage(content=system_message), HumanMessage(content=inputs[0]), AIMessage(content=str(outputs[0])), HumanMessage(content=inputs[2])]

output = chat.invoke(messages)

print(output)




content="[{'payer': 'F.R.J. Terry', 'payee': 'Freedmen', 'amount': 10, 'pay frequency': 'monthly', 'description': 'working in Beagley\\'s Yard for two months commencing on the 14th day of August 1865 and terminating on the 14th day of October 1865. Includes free clothing, good and sufficient quarters, and kind and humane treatment. Labor not to exceed ten hours per day in summer and nine hours in winter.'}]"


## Task #2a: Grab Labor Contract Rows from Kaggle

In [54]:
import pandas as pd

df = pd.read_csv('./input/contract-records.csv')
labor = df[df['sub_category']=='Labor Contracts']
labor.head()